# EthicML

## Runnning experiments on the Adult dataset

### Installation

First we need to install EthicML. Currently, the toolkit isn't on PyPi, but this will change soon.

For now, the toolkit has to be cloned, then installed as an editable package
```
cd <Location to clone to>
git clone https://github.com/predictive-analytics-lab/EthicML.git
cd EthicML
pip install --editable ./
```

(Obviously this notebook is within the package, so we can skip this step)

### Loading the data

EthicML includes some often used datasets from fairness literature.
First, we load one of these... in this example we load the UCI Adult dataset

In [2]:
import ethicml as em

data: em.DataTuple = em.adult().load()
assert (45222, 101) == data.x.shape
assert (45222, 1) == data.s.shape
assert (45222, 1) == data.y.shape

This loads the dataset as a DataTuple, which comprises $x$ (features), $s$ (sensitive attribute and $y$ (class label). Each member of the DataTuple is stored as a Pandas DataFrame.

By default, the Adult dataset uses the binary attribute `sex_Male` as the sensitive feature.

In [3]:
data.s.head()

,sex_Male
0,1
1,1
2,1
3,1
4,0


If we want to run experiments using race as the sensitive attribute we could change that manually, or, as this is a common task, EthicML can split the data for you.

In [4]:
data: em.DataTuple = em.adult(split="Race").load()
assert (45222, 98) == data.x.shape
assert (45222, 1) == data.s.shape
assert (45222, 1) == data.y.shape

In [5]:
data.s.head()

,race
0,4
1,4
2,4
3,2
4,4


However, we're going to be repeating some of the experiments from FairGP. In that paper they do experiments with race as the sensitive attribute, but the value is binary. The value of race is White or Not_White.

Fortunately, race has been one-hot-encoded so to replicate this we can just drop the features from the sensitive attribute that aren't `race_White`.

The Dataset class is really just a guide that tells EthicML how to read the underlying CSV. So to remove the other race attributes, we can just not include them in our list of sensitive attribute columns.

In [6]:
from dataclasses import replace

dataset = em.adult("Race")
dataset = replace(dataset, sens_attr_spec="race_White")
data = em.load_data(dataset)

In [7]:
data.s.head()

,race_White
0,1
1,1
2,1
3,0
4,1


### Evaluating some models

In [8]:
datasets = [dataset, em.toy()]
preprocess_models = [em.Upsampler()]
# inprocess_models = [em.Agarwal(), em.Kamishima(), em.LR(), em.SVM(kernel='linear'), em.Kamiran()]
inprocess_models = [em.LR(), em.SVM(kernel='linear'), em.Kamiran()]
postprocess_models = []
metrics = [em.Accuracy(), em.CV()]
per_sens_metrics = [em.Accuracy(), em.TPR(), em.ProbPos()]

In [9]:
test123 = em.evaluate_models(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=2)

100%|██████████| 28/28 [00:11<00:00,  2.42it/s, model=Kamiran & Calders LR, dataset=Toy, transform=Upsample uniform, repeat=1]


In [10]:
test123

Accuracy  \
dataset    transform        model                       split_id             
Adult Race no_transform     Logistic Regression, C=1.0  0         0.851410   
                            SVM                         0         0.851962   
                            Kamiran & Calders LR        0         0.850525   
                            Logistic Regression, C=1.0  0         0.851410   
                            SVM                         0         0.851962   
...                                                                    ...   
Toy        Upsample uniform SVM (linear)                0         0.950000   
                            Kamiran & Calders LR        0         0.975000   
                            Logistic Regression (C=1.0) 1         1.000000   
                            SVM (linear)                1         1.000000   
                            Kamiran & Calders LR        1         1.000000   

                                                                  Accuracy_race_White_0  \
dataset    transform        model                       split_id                          
Adult Race no_transform     Logistic Regression, C=1.0  0                      0.897538   
                            SVM                         0                      0.898332   
                            Kamiran & Calders LR        0                      0.895155   
                            Logistic Regression, C=1.0  0                      0.897538   
                            SVM                         0                      0.898332   
...                                                                                 ...   
Toy        Upsample uniform SVM (linear)                0                           NaN   
                            Kamiran & Calders LR        0                           NaN   
                            Logistic Regression (C=1.0) 1                           NaN   
                            SVM (linear)                1                           NaN   
                            Kamiran & Calders LR        1                           NaN   

                                                                  Accuracy_race_White_0-race_White_1  \
dataset    transform        model                       split_id                                       
Adult Race no_transform     Logistic Regression, C=1.0  0                                   0.053587   
                            SVM                         0                                   0.053868   
                            Kamiran & Calders LR        0                                   0.051846   
                            Logistic Regression, C=1.0  0                                   0.053587   
                            SVM                         0                                   0.053868   
...                                                                                              ...   
Toy        Upsample uniform SVM (linear)                0                                        NaN   
                            Kamiran & Calders LR        0                                        NaN   
                            Logistic Regression (C=1.0) 1                                        NaN   
                            SVM (linear)                1                                        NaN   
                            Kamiran & Calders LR        1                                        NaN   

                                                                  Accuracy_race_White_0/race_White_1  \
dataset    transform        model                       split_id                                       
Adult Race no_transform     Logistic Regression, C=1.0  0                                   0.940295   
                            SVM                         0                                   0.940036   
                            Kamiran & Calders LR        0                                   0.9420

In [11]:
test123.groupby(level=[0, 1, 2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_race_White_0/race_White_1', 'TPR_race_White_0/race_White_1', 'prob_pos_race_White_0/race_White_1']]

Accuracy            \
                                                             mean       std   
dataset    transform        model                                             
Adult Race Upsample uniform Kamiran & Calders LR         0.846282  0.004343   
                            Logistic Regression (C=1.0)  0.846103  0.004596   
                            Logistic Regression, C=1.0   0.846213  0.004472   
                            SVM                          0.845357  0.005522   
                            SVM (linear)                 0.845384  0.005553   
           no_transform     Kamiran & Calders LR         0.847626  0.003439   
                            Logistic Regression (C=1.0)  0.847485  0.004532   
                            Logistic Regression, C=1.0   0.847651  0.005249   
                            SVM                          0.848557  0.004663   
                            SVM (linear)                 0.847706  0.004915   
Toy        Upsample uniform Kamiran & Calders LR         0.928750  0.063626   
                            Logistic Regression (C=1.0)  0.987500  0.014434   
                            Logistic Regression, C=1.0   0.870000  0.005774   
                            SVM                          0.868750  0.007217   
                            SVM (linear)                 0.975000  0.028868   
           no_transform     Kamiran & Calders LR         0.919375  0.058106   
                            Logistic Regression (C=1.0)  0.968750  0.036084   
                            Logistic Regression, C=1.0   0.873750  0.012990   
                            SVM                          0.872500  0.014434   
                            SVM (linear)                 0.968750  0.021651   

                                                        Accuracy_race_White_0/race_White_1  \
                                                                                      mean   
dataset    transform        model                                                            
Adult Race Upsample uniform Kamiran & Calders LR                                  0.943993   
                            Logistic Regression (C=1.0)                           0.943158   
                            Logistic Regression, C=1.0                            0.942541   
                            SVM                                                   0.942112   
                            SVM (linear)                                          0.942148   
           no_transform     Kamiran & Calders LR                                  0.942877   
                            Logistic Regression (C=1.0)                           0.939919   
                            Logistic Regression, C=1.0                            0.942011   
                            SVM                                                   0.940585   
                            SVM (linear)                                          0.940723   
Toy        Upsample uniform Kamiran & Calders LR                                       NaN   
                            Logistic Regression (C=1.0)                                NaN   
                            Logistic Regression, C=1.0                                 NaN   
                            SVM                                                        NaN   
                            SVM (linear)                                               NaN   
           no_transform     Kamiran & Calders LR                                       NaN   
                            Logistic Regression (C=1.0)                                NaN   
                            Logistic Regression, C=1.0                                 NaN   
                            SVM                                                        NaN   
                            SVM (linear)                                               NaN   

                                                                   \
       

In [12]:
datasets = [em.adult()]
preprocess_models = []
inprocess_models = [em.Agarwal(), em.Kamishima(), em.LR(), em.SVMAsync()]
postprocess_models = []
metrics = [em.Accuracy(), em.CV()]
per_sens_metrics = [em.Accuracy(), em.TPR(), em.ProbPos()]
test123 = await em.evaluate_models_async(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=10, max_parallel=3)

synchronous algorithms...


100%|██████████| 10/10 [00:04<00:00,  2.46it/s, model=Logistic Regression (C=1.0), dataset=Adult Sex - Train (9)]

asynchronous algorithms...



100%|██████████| 30/30 [16:53<00:00, 33.78s/it, model=SVM, dataset=Adult Sex - Train (9), worker_id=1]


In [13]:
test123.groupby(level=[0, 1, 2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_sex_Male_0/sex_Male_1', 'TPR_sex_Male_0/sex_Male_1', 'prob_pos_sex_Male_0/sex_Male_1']]


Accuracy            \
                                                        mean       std   
dataset   transform    model                                             
Adult Sex no_transform Agarwal LR                   0.846136  0.003351   
                       Agarwal, LR, DP              0.845900  0.002629   
                       Kamishima                    0.848135  0.003341   
                       Logistic Regression (C=1.0)  0.847556  0.003511   
                       Logistic Regression, C=1.0   0.846965  0.003432   
                       SVM                          0.861606  0.002751   

                                                   Accuracy_sex_Male_0/sex_Male_1  \
                                                                             mean   
dataset   transform    model                                                        
Adult Sex no_transform Agarwal LR                                        0.881397   
                       Agarwal, LR, DP                                   0.879809   
                       Kamishima                                         0.881765   
                       Logistic Regression (C=1.0)                       0.880436   
                       Logistic Regression, C=1.0                        0.880456   
                       SVM                                               0.896412   

                                                              \
                                                         std   
dataset   transform    model                                   
Adult Sex no_transform Agarwal LR                   0.006163   
                       Agarwal, LR, DP              0.006846   
                       Kamishima                    0.006238   
                       Logistic Regression (C=1.0)  0.005718   
                       Logistic Regression, C=1.0   0.005667   
                       SVM                          0.003890   

                                                   TPR_sex_Male_0/sex_Male_1  \
                                                                        mean   
dataset   transform    model                                                   
Adult Sex no_transform Agarwal LR                                   0.952476   
                       Agarwal, LR, DP                              0.942288   
                       Kamishima                                    0.824512   
                       Logistic Regression (C=1.0)                  0.828655   
                       Logistic Regression, C=1.0                   0.839254   
                       SVM                                          0.744793   

                                                              \
                                                         std   
dataset   transform    model                                   
Adult Sex no_transform Agarwal LR                   0.035648   
                       Agarwal, LR, DP              0.044544   
                       Kamishima                    0.050735   
                       Logistic Regression (C=1.0)  0.055215   
                       Logistic Regression, C=1.0   0.049004   
                       SVM                          0.042370   

                                                   prob_pos_sex_Male_0/sex_Male_1  \
                                                                             mean   
dataset   transform    model                                                        
Adult Sex no_transform Agarwal LR                                        0.400370   
                       Agarwal, LR, DP                                   0.388597   
                       Kamishima                                         0.307629   
                       Logistic Regression (C=1.0)                       0.307150   
                       Logistic Regression, C=1.0                        0.315986   
                       SVM                                      